# How to use Mistral's Codestral API
The notebook offers a practical example of integrating the Codestral API into a Python workflow, focusing specifically on the service's FIM and chat completion features.

1. Setting up API access
2. Defining helper functions
3. Using the FIM endpoint
4. Using the chat endpoint
5. Processing API responses

----
## 1. Setting up API access:
   - Importing necessary libraries (requests, json)
   - Loading the API key from a local file

In [6]:
import requests
import json

In [7]:
with open('/Users/manuel/gdrive/_hidden_apikeys/mistral_codestral.txt') as f:
    apikey_codestral = f.readline()

----
## 2. Defining helper functions:
   - `call_chat_endpoint()`: Sends requests to the chat completion endpoint
   - `call_fim_endpoint()`: Sends requests to the Fill in the Middle (FIM) completion endpoint
   - Both functions handle API authentication and error responses

In [8]:
def call_chat_endpoint(data, api_key=apikey_codestral):
    """Calls the Codestral chat completion endpoint.

    Args:
        data (dict): The data to send to the endpoint.
        api_key (str, optional): The API key to use for authorization. Defaults to apikey_codestral.

    Returns:
        dict or str: The response from the API as a dictionary on success, or an error message on failure.
    """

    # URL for the Codestral chat completion endpoint
    url = "https://codestral.mistral.ai/v1/chat/completions"

    # Headers for the request with authorization, content type, and accept format
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    # Send a POST request with data and headers
    response = requests.post(url, headers=headers, data=json.dumps(data))

    # Check successful response status code
    if response.status_code == 200:
        # Return the JSON response data if successful
        return response.json()
    else:
        # Return an error message with status code and response text on failure
        return f"Error: {response.status_code}, {response.text}"

def call_fim_endpoint(data, api_key=apikey_codestral):
    """Calls the Codestral Fill in the Middle completion endpoint.

    Args:
        data (dict): The data to send to the endpoint.
        api_key (str, optional): The API key to use for authorization. Defaults to apikey_codestral.

    Returns:
        dict or str: The response from the API as a dictionary on success, or an error message on failure.
    """

    # URL for the Codestral FIM completion endpoint
    url = "https://codestral.mistral.ai/v1/fim/completions"

   # Headers for the request with authorization, content type, and accept format
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    # Send a POST request with data and headers
    response = requests.post(url, headers=headers, data=json.dumps(data))

    # Check successful response status code
    if response.status_code == 200:
        # Return JSON response data on success
        return response.json()
    else:
        # Return error message on failure (same as call_chat_endpoint)
        return f"Error: {response.status_code}, {response.text}"


----
## 3. Using the FIM endpoint:
We structure a prompt with a prefix and suffix
We send a request to complete a Python function for inserting a node in a linked list

In [9]:
prompt = """
def insertNodeAtPosition(llist, data, position):
"""
suffix = """
    data = 1
    position = 2
    list = [16, 13, 7]
    llist = SinglyLinkedList()
    for n in list:
        llist.insert_node(n)
    llist_head = insertNodeAtPosition(llist.head, data, position)
    print(llist_head)
    """
data = {
    "model": "codestral-latest",
    "prompt": prompt,
    "suffix": suffix,
    "temperature": 0
}

response_fim = call_fim_endpoint(data)
response_fim

{'id': '5329dc539968454890bd103ca09d6c43',
 'object': 'chat.completion',
 'created': 1720131776,
 'model': 'codestral-latest',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '    new_node = SinglyLinkedListNode(data)\n    if position == 0:\n        new_node.next = llist\n        return new_node\n    else:\n        current_node = llist\n        for i in range(position - 1):\n            current_node = current_node.next\n        new_node.next = current_node.next\n        current_node.next = new_node\n        return llist',
    'tool_calls': None},
   'finish_reason': 'stop',
   'logprobs': None}],
 'usage': {'prompt_tokens': 106,
  'total_tokens': 207,
  'completion_tokens': 101}}

----
## 4. Using the chat endpoint:
We request a function to access a specific key from the result from earlier `response_fim`.

In [10]:
prompt = f"""
Given the following json, write function to access `contents` key from the following json:
{response_fim}
"""
data = {
    "model": "codestral-latest",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "temperature": 0
}

call_chat_endpoint(data)

{'id': '1131612a28a64f479b1793e6e69ef69c',
 'object': 'chat.completion',
 'created': 1720131777,
 'model': 'codestral-latest',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "Here is a Python function that can be used to access the `contents` key from the given JSON:\n\n```python\ndef get_contents(json_data):\n    return json_data['choices'][0]['message']['content']\n\n# Given JSON\njson_data = {\n    'id': '5329dc539968454890bd103ca09d6c43',\n    'object': 'chat.completion',\n    'created': 1720131776,\n    'model': 'codestral-latest',\n    'choices': [\n        {\n            'index': 0,\n            'message': {\n                'role': 'assistant',\n                'content': '    new_node = SinglyLinkedListNode(data)\\n    if position == 0:\\n        new_node.next = llist\\n        return new_node\\n    else:\\n        current_node = llist\\n        for i in range(position - 1):\\n            current_node = current_node.next\\n        new_node.next = 

----
## 5. Processing API responses:
We use the above output to define the `jsonresponse_content()` function, which extracts the generated content from Codestral API responses.

In [11]:
def jsonresponse_content(response):
  """
  Extracts the content from the 'message' key within the first choice of the provided JSON data.

  Args:
      data (dict): The JSON data containing the information.

  Returns:
      str: The content string retrieved from the 'message' key, or None if not found.
  """
  # Check if 'choices' key exists
  if not response['choices'][0]['message']['content']:
    return None
  else:
    return response['choices'][0]['message']['content']

In [12]:
print(jsonresponse_content(response_fim))

    new_node = SinglyLinkedListNode(data)
    if position == 0:
        new_node.next = llist
        return new_node
    else:
        current_node = llist
        for i in range(position - 1):
            current_node = current_node.next
        new_node.next = current_node.next
        current_node.next = new_node
        return llist
